![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and `mistralai/mistral-large` to make simple chat conversation and tool calls

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook provides a detailed demonstration of the steps and code required to showcase support for Chat models, including the integration of tools and watsonx.ai models.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The purpose of this notebook is to demonstrate how to use Chat models, e.g. `mistralai/mistral-large` by using tools.


## Table of Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on watsonx](#models)
- [Work with chat messages](#chat)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the `datasets` and dependencies

In [ ]:
!pip install requests | tail -n 1
!pip install ibm-cloud-sdk-core | tail -n 1

In [1]:
import getpass, os, requests
from ibm_cloud_sdk_core import IAMTokenManager

### Inferencing class
This cell defines a class that makes a REST API call to the watsonx Foundation Model
inferencing API that we will use to generate output from the provided input.
The class takes the access token created in the previous step, and uses it to
make a REST API call with input, model id and model parameters. The response
from the API call is returned as the cell output.


**Action:** Provide watsonx.ai Runtime url to work with watsonx.ai.

In [2]:
endpoint_url = input("Please enter your watsonx.ai Runtime endpoint url (hit enter): ")

Define a `ModelInferanceChat` class for chat generation.

In [3]:
class ModelInferanceChat:
    def __init__(self, access_token, project_id):
        self.access_token = access_token
        self.project_id = project_id

    def chat(
        self, 
        model_id, 
        messages, 
        tools = None, 
        tool_choice = None, 
        tool_choice_option = None,
        **kwargs
    ):
        wml_url = f"{endpoint_url}/ml/v1/text/chat?version=2024-10-07"
        Headers = {
            "Authorization": "Bearer " + self.access_token,
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        data = {
            "model_id": model_id,
            "messages": messages,
            "tools": tools,
            "tool_choice": tool_choice,
            "tool_choice_option": tool_choice_option,
            "project_id": self.project_id,
        }
        data = data | kwargs
        response = requests.post(wml_url, json=data, headers=Headers)
        if response.status_code == 200:
            return response.json()
        else:
            return response.text

### watsonx API connection
Use the code cell below to define the watsonx.ai credentials that are required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">Managing user API keys</a>.

In [4]:
access_token = IAMTokenManager(
    apikey = getpass.getpass("Enter your watsonx.ai api key and hit enter: "),
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

### Defining the project id
You need to provide the project ID to give the Foundation Model the context for the call. If you have a default project ID set in Watson Studio, the notebook obtains that project ID. Otherwise, you need to provide the project ID in the code cell below.

In [5]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="models"></a>
## Set up the Foundation Model on `watsonx.ai`


Specify the `model_id` of the model you will use for the chat with tools.

In [6]:
model_id = "mistralai/mistral-large"

Initialize the `ModelInferanceChat` class.

**Hint:** Your authentication token might expire, if so please regenerate the `access_token` reinitialize the `ModelInferanceChat` class.

In [7]:
model = ModelInferanceChat(access_token, project_id)

<a id="chat"></a>
## Work with chat messages 

#### Working with simple chat message

In [8]:
messages = [
    {
        "role": "user", 
        "content": "What is 1 + 1"
    }
]

simple_chat_response = model.chat(
    model_id=model_id,
    messages=messages
)

In [9]:
print(simple_chat_response["choices"][0]["message"]["content"])

 The sum of 1 + 1 is 2. Here it is:

 1
+1
____
 2


#### Working with simple chat message with params

In [20]:
messages = [
    {
        "role": "user", 
        "content": "What is 1 + 1"
    }
]

simple_chat_response_2 = model.chat(
    model_id=model_id,
    messages=messages,
    temperature=1.5,
    max_tokens=100
)

In [21]:
print(simple_chat_response_2["choices"][0]["message"]["content"])

 The sum of 1 and 1 is 2. So,

1 + 1 = 2


#### Work with an advanced chat message

In [22]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Who won the world series in 2020?"
            }
        ],
    },
    {
        "role": "assistant",
        "content": "The Los Angeles Dodgers won the World Series in 2020."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Where was it played?"
            }
        ],
    }
]

advanced_chat_response = model.chat(
    model_id=model_id,
    messages=messages
)

In [23]:
print(advanced_chat_response["choices"][0]["message"]["content"])

 The 2020 World Series was played at Globe Life Field in Arlington, Texas. Due to the COVID-19 pandemic, the entire series was played at a neutral site for the first time in Major League Baseball history. The Dodgers defeated the Tampa Bay Rays in six games to win their seventh World Series championship.


#### Work with chat messages using `tools` and `tool_choice`

In [24]:
messages = [
    {"role": "user", "content": "What's the weather like in Boston today?"},
]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        },
    }
]

tool_choice = {"type": "function", "function": {"name": "get_current_weather"}}

tool_response = model.chat(
    model_id=model_id,
    messages=messages, 
    tools=tools, 
    tool_choice=tool_choice
)

In [25]:
import json

print(json.dumps(tool_response["choices"][0]["message"], indent=4))

{
    "role": "assistant",
    "content": "",
    "tool_calls": [
        {
            "id": "chatcmpl-tool-6846b1bfab404019ae72b9da50c5e1be",
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "arguments": "{\"location\":\"\"}"
            }
        }
    ]
}


#### Work with chat messages using `tools` and `tool_choice_option`

In [26]:
tools = [
    {
        'function': {
            'description': 'Adds a and b.',
            'name': 'add',
            'parameters': {
                'properties': {
                    'a': {'type': 'float'},
                    'b': {'type': 'float'}},
                'required': ['a', 'b'],
                'type': 'object'}
        },
        'type': 'function'
    },
    {
        'function': {
            'description': 'Multiplies a and b.',
            'name': 'multiply',
            'parameters': {
                'properties': {
                    'a': {'type': 'float'},
                    'b': {'type': 'float'}},
                'required': ['a', 'b'],
                'type': 'object'}
        },
        'type': 'function'
    }
]

tool_choice_option = "auto"

In [27]:
messages = [
    {"role": "user", "content": "What is 5 + 6?"},
]

tools_response = model.chat(
    model_id=model_id,
    messages=messages, 
    tools=tools, 
    tool_choice_option=tool_choice_option
)

print(json.dumps(tools_response["choices"][0]["message"], indent=4))

{
    "role": "assistant",
    "tool_calls": [
        {
            "id": "chatcmpl-tool-268dd36350cd4729b41fc7048b208eec",
            "type": "function",
            "function": {
                "name": "add",
                "arguments": "{\"a\": 5, \"b\": 6}"
            }
        }
    ]
}


In [28]:
messages = [
    {"role": "user", "content": "What is 5 * 6?"},
]

tools_response_2 = model.chat(
    model_id=model_id,
    messages=messages, 
    tools=tools, 
    tool_choice_option=tool_choice_option
)

print(json.dumps(tools_response_2["choices"][0]["message"], indent=4))

{
    "role": "assistant",
    "tool_calls": [
        {
            "id": "chatcmpl-tool-b178a5f6d8f7402b922393dda87e3d27",
            "type": "function",
            "function": {
                "name": "multiply",
                "arguments": "{\"a\": 5, \"b\": 6}"
            }
        }
    ]
}


<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

You learned how to work with chat models using tools and watsonx.ai.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Mateusz Szewczyk**, Software Engineer at watsonx.ai.

Copyright © 2024-2025 IBM. This notebook and its source code are released under the terms of the MIT License.